In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import AveragePooling2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

C:\Users\newuser\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [21]:
from keras.applications.mobilenet_v2 import *
from pyimagesearch.lenet import LeNet
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

In [3]:
imagePaths = list(paths.list_images("dataset"))
data = []
labels = []

In [4]:
for imagePath in imagePaths:

    label = imagePath.split(os.path.sep)[-2]
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    data.append(image)
    labels.append(label)


data = np.array(data, dtype="float32")
labels = np.array(labels)

In [5]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [6]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,
    test_size=0.20, stratify=labels, random_state=42)

In [31]:
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

In [32]:
baseModel = MobileNetV2(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

C:\Users\newuser\Anaconda3\lib\site-packages\keras_applications\mobilenet_v2.py:306: UserWarning: MobileNet shape is undefined. Weights for input shape(224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


In [33]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

In [34]:
model1 = LeNet.build(width=224, height=224, depth=3, classes=2)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
    metrics=["accuracy"])

In [35]:
model = Model(inputs=baseModel.input, outputs=headModel)

In [36]:
for layer in baseModel.layers:
    layer.trainable = False

In [37]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
    metrics=["accuracy"])

In [40]:
H = model.fit_generator(
    aug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)

Epoch 1/20
34/34 [==============================] - 1102s 32s/step - loss: 0.5917 - acc: 0.6857 - val_loss: 0.3274 - val_acc: 0.8804
Epoch 2/20
34/34 [==============================] - 1016s 30s/step - loss: 0.3848 - acc: 0.8168 - val_loss: 0.2128 - val_acc: 0.9203
Epoch 3/20
34/34 [==============================] - 1004s 30s/step - loss: 0.2826 - acc: 0.8741 - val_loss: 0.1585 - val_acc: 0.9529
Epoch 4/20
34/34 [==============================] - 1006s 30s/step - loss: 0.2281 - acc: 0.9170 - val_loss: 0.1182 - val_acc: 0.9674
Epoch 5/20
34/34 [==============================] - 1007s 30s/step - loss: 0.1989 - acc: 0.9161 - val_loss: 0.0836 - val_acc: 0.9710
Epoch 6/20
34/34 [==============================] - 1006s 30s/step - loss: 0.1826 - acc: 0.9281 - val_loss: 0.0735 - val_acc: 0.9710
Epoch 7/20
34/34 [==============================] - 1237s 36s/step - loss: 0.1652 - acc: 0.9406 - val_loss: 0.0650 - val_acc: 0.9710
Epoch 8/20
34/34 [==============================] - 1026s 30s/step - 

In [41]:
predIdxs = model.predict(testX, batch_size=BS)

In [42]:
predIdxs = np.argmax(predIdxs, axis=1)

In [43]:
print(classification_report(testY.argmax(axis=1), predIdxs,
    target_names=lb.classes_))

              precision    recall  f1-score   support

   with_mask       0.97      1.00      0.99       138
without_mask       1.00      0.97      0.99       138

 avg / total       0.99      0.99      0.99       276



In [45]:
model.save("maskmodel")

In [56]:
imgPaths = list(paths.list_images("jei"))
data = []
labels = []
for imgPath in imgPaths:
    label = imgPath.split(os.path.sep)[-2]
    image = load_img(imgPath, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    data.append(image)
    labels.append(label)

data = np.array(data, dtype="float32")
labels = np.array(labels)

In [57]:
(mask, withoutMask) = model.predict(data)[0]

In [50]:
mask

0.2434639

In [52]:
withoutMask

0.75653607